# Collaborative Filtering

Pada code ini akan dilakukan colaborative filtering (user-based nearest neighbor) dari data pemberian rating oleh 24 customer pada 10 film yang ada pada dataset.
<br>
<br>
Analisis ini ditujukan untuk:
1. Menentukan rekomendasi film kepada customer tertentu terhadap film-film yang belum ditonton. Dalam hal ini ketika customer tidak memberikan rating pada film maka diasumsikan bahwa customer tersebut belum menonton film tersebut.
2. Mengetahui hubungan antar dua orang customer berdasarkan kemiripan rating yang diberikan kepada masing-masing film pada dataset.
3. Mengetahui hubungan antar dua orang customer berdasarkan kemiripan film-film yang sudah ditonton dan belum ditonton.

Dataset yang digunakan terdiri atas 11 features, yaitu nama customer dan rating yang diberikan kepada masing-masing film terpilih yang terdiri atas:
1. Ada Apa dengan Cinta 2
2. Gundala
3. Dilan 1991
4. Bumi Manusia 
5. Dua Garis Biru
6. Avengers: End Game
7. The Lion King
8. Aladdin
9. Spiderman: Far From Home
10. Captain Marvel
<br>

## Import Libraries

In [1]:
# import Libraries
# ambil dari dataset adsb_rating untuk import dataset movie ratings
import pandas as pd 
import numpy as np
from adsb_rating import dataset
from math import sqrt

## Load Data

In [4]:
df_rating = pd.read_csv('../data/ratings.csv')
df_rating.head()

,customer,movie_id,rating
0,Aika,1001,3.0
1,Aika,1002,5.0
2,Aika,1003,4.0
3,Aika,1004,4.0
4,Aika,1005,4.0


In [5]:
df_movies = pd.read_csv('../data/moviescode.csv')
df_movies.head()

,movie_id,movie
0,1001,Ada Apa dengan Cinta 2
1,1002,Gundala
2,1003,Dilan 1991
3,1004,Bumi Manusia
4,1005,Dua Garis Biru


In [6]:
df = pd.merge(df_rating, df_movies, on='movie_id')
df.head()

,customer,movie_id,rating,movie
0,Aika,1001,3.0,Ada Apa dengan Cinta 2
1,Bika,1001,NaN,Ada Apa dengan Cinta 2
2,Cika,1001,NaN,Ada Apa dengan Cinta 2
3,Dika,1001,4.0,Ada Apa dengan Cinta 2
4,Eika,1001,3.0,Ada Apa dengan Cinta 2


In [7]:
df.groupby('movie')['rating'].mean().sort_values(ascending=False).round(1)

movie
Avengers: End Game          4.7
Bumi Manusia                4.5
Aladdin                     4.4
Spiderman: Far From Home    4.4
The Lion King               4.3
Captain Marvel              4.0
Ada Apa dengan Cinta 2      3.9
Gundala                     3.9
Dua Garis Biru              3.8
Dilan 1991                  3.7
Name: rating, dtype: float64

In [8]:
df.groupby('movie')['rating'].count().sort_values(ascending=False).round(1)

movie
Avengers: End Game          19
Captain Marvel              18
Ada Apa dengan Cinta 2      17
Spiderman: Far From Home    16
Dilan 1991                  14
Aladdin                     14
Gundala                     13
The Lion King               11
Dua Garis Biru               9
Bumi Manusia                 6
Name: rating, dtype: int64

**Movies Analysis** <br>
Dari data di atas terlihat bahwa film yang paling banyak ditonton adalah Avengers: End Game dan film yang ratingnya paling bagus adalah Avengers: End Game. Sehingga dapat disimpulkan bahwa Avengers: End Game dapat direkomendasikan untuk seluruh customer yang belum pernah menonton. Karena peluang customer tersebut akan suka dengan Avengers: End Game sangat besar.

## User Recommendations

Berikut adalah code yang digunakan untuk menentukan rekomendasi seluruh film yang belum ditonton oleh seorang customer diurutkan berdasarkan rating yang paling tinggi. 

In [9]:
# rekomendasi film yang belum ditonton kepada seorang customer diurutkan berdasarkan rating tertinggi
def user_recommendations(person):

    # memberikan rekomendasi film berdasarkan rata-rata rating yang diberikan oleh customer lainnya kepada masing-masing film
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # rating yang digunakan untuk menentukan rekomendasi kepada customer, mengabaikan rating dari customer itu sendiri
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # mengabaikan jika korelasi pearson antara customer tersebut dengan customer lain bernilai 0
        if sim <=0: 
            continue
        for item in dataset[other]:

            # hanya menggunakan film-film yang sudah sama-sama ditonton oleh customer tersebut dengan customer lain
            if item not in dataset[person] or dataset[person][item] == 0:

            # total dari seluruh kolerasi pearson antara customer tersebut dengan customer lainnya
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # jumlah dari seluruh korelasi pearson dari customer tersebut dengan customer lain
                simSums.setdefault(item,0)
                simSums[item]+= sim

    # score yang digunakan untuk pemberian rekomendasi film adalah
    # normalisasi dari jumlah seluruh rating yang diberikan oleh customer lain pada masing-masing film yang belum ditonton oleh customer tersebut
    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    # mengurutkan normalisasi rating dari yang tertinggi sampai dengan terendah
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    return recommendataions_list

## Similarity Score

Berikut adalah code yang digunakan untuk menentukan seberapa mirip rating yang diberikan untuk seluruh film oleh 2 orang customer.

In [10]:
# hubungan pemberian score antara customer 1 dan customer 2 terhadap masing-masing film
def similarity_score(person1,person2):

    # menentukan kesamaan pemberian score terhadap masing-masing film
    # dengan menggunakan euclidean distance antara customer 1 dan customer 2 

    # Jika customer 1 dan customer 2 sama-sama telah menonton film tertentu maka akan diberikan nilai 1
    both_viewed = {}

    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
        
        # Jika customer 1 dan customer 2 sama-sama belum menonton film tertentu maka akan diberikan nilai 0
        if len(both_viewed) == 0:
            return 0

        # menentukan euclidean distance
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
        
        return 1/(1+sqrt(sum_of_eclidean_distance))

## Person Correlation

Berikut ada code yang digunakan untuk menentukan seberapa mirip kesukaan film antara 2 customer berdasarkan film-film yang telah ditonton oleh keduanya.

In [11]:
# hubungan antara customer 1 dan customer 2 terhadap film-film yang sudah ditonton
def person_correlation(person1, person2):

   # Jika customer 1 dan customer 2 sama-sama telah menonton film tertentu maka akan diberikan nilai 1
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Jika customer 1 dan customer 2 sama-sama belum menonton film tertentu maka akan diberikan nilai 0
    if number_of_ratings == 0:
        return 0

    # menjumlahkan seluruh preferensi film-film yang telah ditonton oleh customer 1 dan customer 2
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # menjumlahkan dari kuadrat masing-masing preferensi film-film yang telah ditonton oleh customer 1 dan customer 2
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # menjumlahkan hasil kali dari preferensi kedua customer untuk masing-maasing film
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # mencari korelasi antara customer 1 dan customer 2 berdasarkan film-film yang sudah pernah ditonton
    numerator_value = product_sum_of_both_users - (person1_preferences_sum * person2_preferences_sum / number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2) / number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

## Person Correlation with All Others Customer

Berikut adalah code yang digunakan untuk melihat kesamaan selera film yang telah ditonton oleh seorang customer dengan customer lainnya diurutkan dari kesamaan yang paling dekat dengan customer tersebut.

In [12]:
def person_correlation_toall(person, number_of_users):

    # korelasi antara customer 1 dan customer 2
    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]

    # mengurutkan korelasi antara customer 1 dan customer 2 pada masing-masing pasangan yang muncul
    # diurutkan dari nilai korelasi tertinggi
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

## Example Case

Memberikan rekomendasi untuk Oika dan Kika serta mengetahui seberapa dekat kesamaan diantara keduanya.

In [13]:
# peluang kesamaan pemberian rating pada masing-masing film oleh Oika dan Kika
print(similarity_score('Oika','Kika'))

0.18660549686337075


Nilai peluang pemberian rating yang sama untuk suatu film tertentu oleh Oika dan Kika hanya 0.19. Artinya standard baik/tidaknya pun bagus/buruknya suatu film berbeda jauh. Jadi misalkan bagi Oika film tersebut bagus pun baik, namun bagi Kika film tersebut tidak bagus pun tidak baik dan berlaku sebaliknya.

In [14]:
# seberapa erat kesamaan antara Oika dan Kika terhadap selera film berdasarkan film-film yang sudah ditonton oleh keduanya
print(person_correlation('Bika', 'Jika'))

0.0652546122079844


Nilai korelasi pearson antara Oika dan Kika mencapai 0.78. Artinya dapat dikatakan bahwa film-film yang disukai oleh Oika kemungkinan besar disukai juga oleh Kika dan berlaku sebaliknya. Sehingga jika film tersebut ditonton oleh Oika kemungkinan besar Kika juga menontonnya dan berlaku sebaliknya. Dengan hubungan ini maka dapat memberikan rekomendasi untuk nonton bersama antara Oika dan Kika karena peluang sama-sama suka terhadap suatu film yang dipilih oleh salah satu diantara mereka cukup tinggi.

In [15]:
# rekomendasi film untuk Oika dari keseluruhan film yang berlum ditonton diurutkan berdasarkan score tertinggi
print(user_recommendations('Oika'))

['Dua Garis Biru', 'Bumi Manusia', 'Ada Apa dengan Cinta 2']


Film yang belum ditonton oleh Oika dari 10 film tersedia ada tiga, yaitu AADC2, Dua Garis Biru, dan Bumi Manusia. berdasarkan behavior dari customer lainnya yang telah menonton film-film tersebut maka dapat memberikan rekomendasi kepada Oika untuk menonton AADC2 sebab pada kelompok orang sekitar film tersebut memberikan kesan yang baik yang dibuktikan dengan rating tertinggi dari tiga pilihan film.

In [16]:
# rekomendasi film untuk Kika dari keseluruhan film yang berlum ditonton diurutkan berdasarkan score tertinggi
print(user_recommendations('Kika'))

['Dua Garis Biru', 'Bumi Manusia']


Film yang belum ditonton oleh Kika dari 10 film tersedia ada dua, yaitu Dua Garis Biru, dan Bumi Manusia. berdasarkan behavior dari customer lainnya yang telah menonton film-film tersebut maka dapat memberikan rekomendasi kepada Kika untuk menonton Dua Garis Biru sebab pada kelompok orang sekitar film tersebut memberikan kesan yang baik yang dibuktikan dengan rating tertinggi dari dua pilihan film.

In [ ]:
# melihat customer mana yang memiliki kesamaan paling erat dengan Oika berdasarkan preferensi film yang sudah ditonton oleh keduanya
print(person_correlation_toall('Oika', 24))

Dari list korelasi pearson antara Oika dan customer lainnya dapat dilihat bahwa korelasi terbesar adalah antara Oika dengan Kika, yaitu senilai 0.78. Sehingga dapat disimpulkaln bahwa Oika dan Kika memiliki selera serupa dalam hal film. Sesuai dengan yang telah disebutkan di atas bahwa kedua akan memiliki kesan yang baik ketika menonton bersama. Film yang dipilih oleh Oika kemungkinan besar akan disukai oleh Kika, begitu pula sebaliknya.

## Conclusion

Dari hasil collaborative analysis di atas terdapat 2 hal yang dapat menjadi rekomendasi ataupun improvement:
1. Memberikan rekomendasi film untuk ditonton oleh seorang customer dengan melihat behaviour dari kelompok customer lainnya selain dirinya sendiri. Dalam hal ini dilihat film manakah yang paling diminati oleh customer lainnya, maka film tersebut yang akan direkomendasikan dengan customer.
2. Memberikan rekomendasi pasangan nonton bersama dengan melihat kesamaan selera antar dua pelanggan berdasarkan film-film yang telah ditonton oleh masing-masing dari mereka. Jika film yang telah ditonton oleh keduanya mayoritas sama (korelasi > 0.5) maka keduanya dapat direkomendasikan untuk nonton bersama.